<a href="https://colab.research.google.com/github/abdulrahmanalaa123/Graduation_Project/blob/Honda/tsfel_sha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pickle5
!pip install tsfel
import pickle5 as pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn 
import pandas as pd
import json
import tensorflow as tf
import tsfel as tsfel
from re import L
from operator import itemgetter
from google.colab import files
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras import models   # for model loading
from sklearn.model_selection import GridSearchCV    
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from statsmodels.tsa.stattools import adfuller

#%pip install pandas==1.4.1       unpickle problem
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

with open("/content/drive/MyDrive/Colab Notebooks/hr(tuples).pkl", "rb") as f: 
    data = pickle.load(f)

In [ ]:
print(data)

In [20]:
arr = np.array([data["Participant"]])
print(arr)

[[22 22 22 ... 64 64 64]]


In [23]:
data[data["Statge"]]

SyntaxError: ignored

In [26]:
def evensampler(data):
    mask = data["Stage"] == "STIMULUS"
    participants = data["Participant"].unique()
    trainval = int(0.9*len(participants))
    trainpar = participants[:trainval]
    testpar = participants[trainval:]
    masktrain = data["Participant"].isin(trainpar)
    masktest = data["Participant"].isin(testpar)
    train = data.loc[mask&masktrain,data.columns[2:-13]].values
    traintarget = data.loc[mask&masktrain,"Target"].values
    test = data.loc[mask&masktest,data.columns[2:-13]].values
    testtarget = data.loc[mask&masktest,"Target"].values

    return train,test,traintarget,testtarget
def getter(x):
    if isinstance(x,tuple):
      
        return itemgetter(0)(x)
    else:
        return x
def splittuple(dat):
  return dat.applymap(lambda x:getter(x))

hrdata=splittuple(data)
xtrain1,xtest1,ytrain1,ytest1 = evensampler(hrdata)
print(xtrain1.T)
cols=hrdata.columns[2:-13]
X_train_sig = pd.DataFrame(xtrain1.T, index=[cols])
Tx_train=X_train_sig
#Tx_train = Tx_train.replace(0, np.nan)
print(Tx_train)

[[87 81 85 ... 77 76 84]
 [87 81 85 ... 77 76 84]
 [87 81 85 ... 76 76 84]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
                       0    1    2    3    4    5    6    7    8    9    ...  \
0:0.0                   87   81   85   85   83   78   79   78   79   77  ...   
0:0.099791              87   81   85   86   83   78   78   77   79   77  ...   
0:0.199582              87   81   85   85   83   78   78   78   79   77  ...   
0:0.299373              87   81   85   86   83   78   78   78   79   77  ...   
0:0.399164              87   81   85   85   83   78   78   78   79   77  ...   
...                    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
10:27.186435000000074    0    0    0    0    0    0    0    0    0    0  ...   
10:27.286226000000056    0    0    0    0    0    0    0    0    0    0  ...   
10:27.386017000000038    0    0    0    0    0    0    0    0    0    0  ...   
10:27.48580800000002     0    0    0    0   

In [7]:
print(type(Tx_train))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
cfg_file = tsfel.get_features_by_domain()                                                        # If no argument is passed retrieves all available features
X_train = tsfel.time_series_features_extractor(cfg_file, Tx_train, fs=50, window_size=50)    # Receives a time series sampled at 50 Hz, divides into windows of size 250 (i.e. 5 seconds) and extracts all features
  



In [28]:
print(Tx_train.loc[:,:2])

                        0   1   2
0:0.0                  87  81  85
0:0.099791             87  81  85
0:0.199582             87  81  85
0:0.299373             87  81  85
0:0.399164             87  81  85
...                    ..  ..  ..
10:27.186435000000074   0   0   0
10:27.286226000000056   0   0   0
10:27.386017000000038   0   0   0
10:27.48580800000002    0   0   0
10:27.585599000000002   0   0   0

[6290 rows x 3 columns]


In [30]:
X_train[]
#it looks bullshit , maybe cuz i didn't change how the data look like as input"inputshape"  or thats how it looks like , try playing around with shape first then find model
#for the classification then change fs,window size ,trial and error and write down the accuracies

,0_Absolute energy,0_Area under the curve,0_Autocorrelation,0_Centroid,0_ECDF Percentile Count_0,0_ECDF Percentile Count_1,0_ECDF Percentile_0,0_ECDF Percentile_1,0_ECDF_0,0_ECDF_1,...,9_Wavelet variance_0,9_Wavelet variance_1,9_Wavelet variance_2,9_Wavelet variance_3,9_Wavelet variance_4,9_Wavelet variance_5,9_Wavelet variance_6,9_Wavelet variance_7,9_Wavelet variance_8,9_Zero crossing rate
0,374996.0,84.87,374996.0,0.487006,10.0,40.0,86.0,87.0,0.02,0.04,...,83.298300,266.529451,517.238090,797.593508,1068.384989,1286.969351,1416.411601,1444.605991,1398.751448,0.0
1,386103.0,86.10,386103.0,0.500414,10.0,40.0,86.0,90.0,0.02,0.04,...,84.712270,274.157674,535.362988,819.164530,1087.584019,1303.319557,1431.090088,1459.252074,1415.473984,0.0
2,412610.0,89.03,412610.0,0.492206,10.0,40.0,90.0,91.0,0.02,0.04,...,86.661140,275.598080,538.367334,829.518899,1101.792923,1314.578607,1436.625128,1461.448054,1418.899294,0.0
3,411878.0,88.95,411878.0,0.490536,10.0,40.0,90.0,91.0,0.02,0.04,...,91.472363,294.995887,581.900323,904.240768,1217.150477,1474.989196,1632.000292,1668.154628,1610.844694,0.0
4,402174.0,87.90,402174.0,0.485537,10.0,40.0,89.0,90.0,0.02,0.04,...,95.872340,307.269463,603.605530,932.220854,1240.028192,1476.180692,1608.359591,1633.181796,1585.357559,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.02,0.04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
121,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.02,0.04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
122,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.02,0.04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
123,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.02,0.04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [31]:
final = X_train.values

In [34]:
new = np.reshape(final,(407,125,159))

In [ ]:
new

In [37]:
new[1][0][3]

0.4870059947306104

In [38]:
X_train["1_Centroid"]

0      0.495408
1      0.488753
2      0.487381
3      0.493108
4      0.493838
         ...   
120    0.000000
121    0.000000
122    0.000000
123    0.000000
124    0.000000
Name: 1_Centroid, Length: 125, dtype: float64

In [41]:
new[0][1][0]

359053.0

In [42]:
X_train['1_Absolute energy']

0      337055.0
1      342635.0
2      333592.0
3      337528.0
4      349144.0
         ...   
120         0.0
121         0.0
122         0.0
123         0.0
124         0.0
Name: 1_Absolute energy, Length: 125, dtype: float64

In [45]:
new = np.reshape(np.transpose(final),(407,125,159))

In [46]:
new

array([[[3.74996000e+05, 3.86103000e+05, 4.12610000e+05, ...,
         8.21300000e+01, 8.08800000e+01, 8.31300000e+01],
        [8.07100000e+01, 7.88400000e+01, 8.20600000e+01, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.48427813e+03, 1.52734333e+03, 1.47874896e+03, ...,
         1.98117509e+03, 1.86561232e+03, 1.87498794e+03],
        [1.77533822e+03, 1.70590157e+03, 1.67135609e+03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[3.59053000e+05, 3.45804000e+05, 3.37685000e+05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 

In [48]:
X_train["405_Entropy"]

0      0.375409
1      0.249063
2      0.322819
3      0.276503
4      0.342787
         ...   
120    0.000000
121    0.000000
122    0.000000
123    0.000000
124    0.000000
Name: 405_Entropy, Length: 125, dtype: float64

In [57]:
def reshaper(data: pd.Series):
  cfg_file = tsfel.get_features_by_domain()                                                        # If no argument is passed retrieves all available features
  X_train = tsfel.time_series_features_extractor(cfg_file,data.T, fs=50, window_size=50)    # Receives a time series sampled at 50 Hz, divides into windows of size 250 (i.e. 5 seconds) and extracts all features
  features = [tuple(row) for row in X_train.values]
  print(len(features),len(features[0]))
  print(features) 
  return features



In [58]:
print(reshaper(hrdata.iloc[1,2:-13]))

*** Feature extraction started ***



*** Feature extraction finished ***
125 159
[(364157.0, 83.64, 364157.0, 0.4923056264193741, 85.0, 85.0, 85.0, 85.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.16386291107994097, 0.0006898846677873525, 0.1790016647788789, 0.0001397578730711326, 0.00013906966498025656, 0.0007366246299274885, 0.000418641808172607, 0.0007094287383323443, 0.0013475786296080314, 0.0008257452003314421, 0.0013705738720513644, 0.001916438377174305, 0.0012458681959563683, 0.02486413849938411, 0.0019243738389357495, 0.002289780784939705, 0.0015718627530752433, 0.002271229747680588, 0.002420561613853947, 0.0008406593705809191, 0.0012909300745898157, 0.0027714295552225413, 0.0030318394971574525, 0.00029401109227902947, 0.0006081362715726875, 0.0004103875370618463, 0.0003172920059537815, 1.0416666666666667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0529413690232107e-05, 1.0, -1.5436720142602693, 0.22139578638531798, 1.2114128108220157, 0.733082534531007, 1.2114128108220157, 0.73308253453